# Imports

In [199]:
import random
import sys
from time import time

import numpy as np
import pandas as pd

# Instructions

Our metric is the Expected Degrees of Separation, EDS, between Shop A and Shop B. That is, imagine a token is given to a customer of Shop A at random. The customer is told to give the token to a customer in the next shop they visit, again at random, and to mark the token. At each step the customer is told to do the same thing, so that the number of marks on the token records the number of customers the token has passed through. If the token comes to Shop B, it is taken, and the number of marks is recorded. EDS is the expected number of marks on the token at the destination shop.

Suppose we have a transition matrix, the probability that a customer Shop A also shops at Shop B.

From Shop A	  |To Shop B       |P(A -> B)
--------------|----------------|--------
Tesco, Bristol|	Boots, Bath	   |0.1
Tesco, Bristol|	Asda, Bristol  |0.2
Asda, Bristol |	Currys, Bristol|0.1

We are looking for a table of the form:

Source Shop A |	Destination Shop B |EDS(A -> … -> B)
--------------|--------------------|-----------------
Tesco, Bristol|	Boots, Bath|3.5
Tesco, Bristol|Asda, Bristol|2.3
Tesco, Bristol|	Currys, Bristol|4.0


Questions we might ask:

- What assumptions are made in the calculation of EDS?
- How would you calculate EDS if the number of shops was small (say <1,000)?
- What challenges would be faced as the number of shops grows? How would you deal with these?
- What are the properties of EDS as a metric?
- Given a matrix of EDS values, how might you estimate the latent shopping preferences of the customers?


# Function definitions 

## Generate transition matrices

In [5]:
def rand_square_array(size=10, sparsify=1.0):
    sq = np.random.exponential(size=(size, size))
    if 0.0 < sparsify < 1.0:
        sq = sq * np.random.binomial(1, sparsify, size=(size, size))
    np.fill_diagonal(sq, 0)
    sq_normed = sq / sq.sum(axis=1)[:,None]
    return sq_normed

In [139]:
def grouped_rand_square_array(side=10, groups=3, group_bonus=5, return_edges=False):
    sq = np.random.exponential(size=(side, side))
    group_edges = [0] + sorted(np.random.choice(range(1, side-1), groups-1, replace=False)) + [side]
    for i in range(groups):
        sq[group_edges[i]:group_edges[i+1], group_edges[i]:group_edges[i+1]] *= group_bonus
    np.fill_diagonal(sq, 0)
    sq_normed = sq / sq.sum(axis=1)[:,None]
    return (sq_normed, group_edges) if return_edges else sq_normed

## Build EDS matrix using mmult and infinite sums

In [3]:
def get_components(M, a, z):
    idx = list(range(M.shape[0]))
    idx.pop(z)
    M_excl_Z = M[idx, :][:, idx]
    M_to_Z = M[:, z][idx]
    initial_state = M[a, :][idx]
    return M_excl_Z, M_to_Z, initial_state

In [164]:
def get_eds_for_all_pairs(M, get_top_left_sq=None, max_steps=100, stop_tol=1e-6, stop_steps=5):
    assert M.shape[0] == M.shape[1], "Matrix not square"
    eds = {}
    dim = get_top_left_sq if get_top_left_sq else M.shape[0]
    
    for a in range(dim):
        eds[a] = {}
        
        for z in range(dim):
            M_excl_Z, M_to_Z, state = get_components(M, a, z)
            results = {}
            results[1] = {'prob': M[a, z], 'cum_prob': M[a, z], 'expected_steps': M[a, z], 'prob_ratio': np.nan}
            tol_steps = 0
            
            for i in range(2, max_steps + 1):
                results[i] = {}
                results[i]['prob'] = state @ M_to_Z
                results[i]['cum_prob'] = results[i-1]['cum_prob'] + results[i]['prob']
                results[i]['expected_steps'] = results[i-1]['expected_steps'] + (results[i]['prob'] * i)
                results[i]['prob_ratio'] = (results[i]['prob'] / results[i-1]['prob'])

                if abs(results[i]['prob_ratio'] - results[i-1]['prob_ratio']) < stop_tol:
                    tol_steps += 1
                    if tol_steps == stop_steps:
                        results['final'] = {}
                        infinite_sum_multiplier = 1 / (1 - results[i]['prob_ratio'])
                        results['final']['prob'] = results[i]['prob'] * results[i]['prob_ratio'] * infinite_sum_multiplier
                        results['final']['cum_prob'] = results[i]['cum_prob'] + results['final']['prob']
                        results['final']['expected_steps'] = (results[i]['expected_steps']
                                                              + i * results['final']['prob']
                                                              + results['final']['prob'] * infinite_sum_multiplier)
                        eds[a][z] = results['final']['expected_steps']
                        break
                else:
                    tol_steps = 0

                state = state @ M_excl_Z
                
    return eds

## Get total memory size of objects

In [204]:
def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

# Scratchpad 

In [140]:
transitions, group_edges = grouped_rand_square_array(side=15, groups=3, group_bonus=20, return_edges=True)
group_edges

[0, 8, 13, 15]

In [141]:
pd.DataFrame(transitions)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,0.290258,0.094889,0.125230,0.132082,0.037280,0.086560,0.168605,0.004555,0.001726,0.002554,0.006313,0.012957,0.019949,0.017042
1,0.222039,0.000000,0.206758,0.085416,0.051718,0.256373,0.037473,0.056954,0.006869,0.019218,0.020019,0.012933,0.008876,0.002774,0.012579
2,0.109621,0.271004,0.000000,0.022360,0.175529,0.008872,0.181700,0.188530,0.024054,0.005274,0.001675,0.000540,0.005180,0.001421,0.004241
3,0.001935,0.131120,0.119390,0.000000,0.263524,0.004748,0.362435,0.068203,0.010666,0.023229,0.006561,0.000014,0.000836,0.003320,0.004019
4,0.218633,0.069162,0.002232,0.003369,0.000000,0.175124,0.310536,0.189623,0.006111,0.002678,0.009696,0.000884,0.003119,0.007693,0.001140
5,0.215256,0.061242,0.107556,0.295733,0.003021,0.000000,0.075074,0.181531,0.001199,0.000606,0.048090,0.005034,0.002735,0.000673,0.002251
6,0.033990,0.078992,0.459098,0.035785,0.058143,0.091533,0.000000,0.062048,0.020995,0.009168,0.044834,0.017999,0.015725,0.042148,0.029540
7,0.165980,0.015786,0.078908,0.111454,0.091807,0.195042,0.293433,0.000000,0.010430,0.003856,0.007661,0.003506,0.013328,0.005497,0.003314
8,0.004464,0.001748,0.000169,0.029089,0.003947,0.000011,0.027886,0.004461,0.000000,0.221285,0.340051,0.107779,0.255159,0.002259,0.001692
9,0.005217,0.000267,0.013169,0.027904,0.000932,0.021340,0.005308,0.006641,0.146271,0.000000,0.404813,0.242966,0.118654,0.004699,0.001819


In [142]:
(transitions > 0).sum()

210

In [143]:
eds_matrix = get_eds_for_all_pairs(transitions, max_steps=100)

/Users/arthurjackrussell/venv/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars


In [144]:
pd.DataFrame(eds_matrix).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,13.037523,10.148326,10.138578,14.804271,14.459518,13.299107,8.635099,11.749667,24.620065,26.924392,22.109486,27.410099,26.058018,76.681237,76.325280
1,10.822722,13.174101,9.542780,14.932209,15.696475,11.326850,9.373798,12.831642,24.317963,26.498927,21.669132,27.111985,25.859619,78.095960,76.707925
2,11.711068,10.340322,10.893859,16.437979,14.059768,13.516683,7.926555,11.443024,24.450120,26.917303,22.075593,27.529584,26.150112,77.917141,77.137486
3,13.329992,12.089968,9.434725,17.498417,12.860118,13.987743,6.315885,12.933330,24.335396,26.502764,21.782472,27.320308,25.999311,77.387102,76.941542
4,10.745651,12.503846,10.495328,16.203184,16.670015,11.898120,6.865538,11.317626,24.698779,27.054153,22.044854,27.606867,26.230712,77.108136,77.137602
5,11.229440,12.461530,10.072089,12.202977,15.783046,14.501156,8.432743,11.621990,24.558984,26.848905,21.621564,27.444197,26.090602,78.097933,77.278530
6,13.410578,12.683773,7.685559,16.982908,16.095012,13.769565,10.285564,13.341616,23.242771,25.681210,20.661203,26.131212,24.828283,75.135080,75.222938
7,11.635549,13.114412,9.936469,14.875655,15.119758,12.044161,6.966004,13.684023,24.442876,26.815733,21.873907,27.361494,25.940887,77.338530,77.042831
8,21.087047,21.186906,19.109825,22.283484,24.650021,19.595852,17.069114,21.132332,12.563446,11.816086,7.345046,13.508127,11.206660,78.806178,74.796575
9,20.910577,21.041389,18.916643,22.135088,24.542504,19.238463,17.100408,20.946344,10.667314,14.941467,7.246451,12.628339,12.631764,78.662190,74.763650


In [133]:
sq_normed = grouped_rand_square_array(15, groups=3, group_bonus=10)
pd.DataFrame(sq_normed).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,0.000893,0.232054,0.038914,0.115348,0.014583,0.012356,0.002237,0.008536,0.004708,0.004876,0.052032,0.066659,0.011821,0.003415
1,0.177749,0.000000,0.100254,0.111573,0.346605,0.033587,0.035255,0.036459,0.032757,0.000918,0.007628,0.007367,0.006787,0.010590,0.024208
2,0.011070,0.030280,0.000000,0.331608,0.178687,0.063196,0.062019,0.001136,0.006626,0.012668,0.015211,0.030323,0.002240,0.010973,0.053101
3,0.250524,0.198763,0.162359,0.000000,0.154083,0.016126,0.047627,0.004648,0.007855,0.015890,0.023235,0.003026,0.008624,0.002193,0.082429
4,0.347398,0.312899,0.224083,0.009123,0.000000,0.065885,0.055115,0.005947,0.010310,0.048490,0.007914,0.041359,0.005195,0.029709,0.098122
5,0.016946,0.017761,0.025619,0.021662,0.009268,0.000000,0.518958,0.002679,0.014916,0.014066,0.018353,0.033923,0.013778,0.019018,0.114096
6,0.003202,0.089545,0.008776,0.008446,0.031808,0.589079,0.000000,0.020713,0.022662,0.006379,0.029129,0.007009,0.041452,0.004436,0.014332
7,0.012280,0.034948,0.065098,0.133400,0.024413,0.021809,0.007814,0.000000,0.130666,0.041866,0.049339,0.228250,0.374401,0.153333,0.111317
8,0.004117,0.007196,0.024172,0.142883,0.001639,0.011345,0.055697,0.165074,0.000000,0.019395,0.240263,0.021525,0.100797,0.148736,0.043284
9,0.026390,0.084469,0.015737,0.128335,0.032922,0.008764,0.080203,0.098468,0.127313,0.000000,0.017072,0.116238,0.152416,0.160855,0.081489


In [183]:
sum([1,2,3,4])

10

In [282]:
%%time
shop_num = 50
shop_list = list(range(shop_num))
transition_matrix = grouped_rand_square_array(shop_num, groups=5, group_bonus=10)
walk = [0]
current_shop = 0
last_visited = np.zeros(shop_num).astype(int)

completed_journeys = {}
for i in range(shop_num):
    completed_journeys[i] = {}
    for j in range(shop_num):
        completed_journeys[i][j] = {'num': 0, 'total_steps': 0, 'sum_of_sqares': 0}

CPU times: user 34.5 ms, sys: 3.09 ms, total: 37.5 ms
Wall time: 38 ms


In [283]:
%%time
num_steps = 300000
for steps in range(1, num_steps):
    if steps % 50000 == 0:
        print(f'{steps} completed.')
    current_shop = np.random.choice(shop_list, p=transition_matrix[current_shop])
    walk.append(current_shop)
    last_occurence = last_visited[current_shop]
    last_visited[current_shop] = steps
    for i in range(last_occurence, steps):
        completed_journeys[walk[i]][current_shop]['num'] += 1
        completed_journeys[walk[i]][current_shop]['total_steps'] += (steps - i)
        completed_journeys[walk[i]][current_shop]['sum_of_sqares'] += (steps - i) ** 2

10000 completed.
20000 completed.
30000 completed.
40000 completed.
50000 completed.
60000 completed.
70000 completed.
80000 completed.
90000 completed.
100000 completed.
110000 completed.
120000 completed.
130000 completed.
140000 completed.
150000 completed.
160000 completed.
170000 completed.
180000 completed.
190000 completed.
200000 completed.
210000 completed.
220000 completed.
230000 completed.
240000 completed.
250000 completed.
260000 completed.
270000 completed.
280000 completed.
290000 completed.
CPU times: user 42.8 s, sys: 195 ms, total: 43 s
Wall time: 43 s


In [301]:
%%time
eds = {'n': {}, 'eds': {}, 'var': {}, 'var_of_sample_mean': {}}
for i in range(shop_num):
    eds['n'][i] = {}
    eds['eds'][i] = {}
    eds['var'][i] = {}
    eds['var_of_sample_mean'][i] = {}
    for j in range(shop_num):
        num_journeys = completed_journeys[i][j]['num']
        total_steps = completed_journeys[i][j]['total_steps']
        sum_of_sqares = completed_journeys[i][j]['sum_of_sqares']
        eds['n'][i][j] = num_journeys
        eds['eds'][i][j] = total_steps / num_journeys
        eds['var'][i][j] = (sum_of_sqares / num_journeys) - (eds['eds'][i][j] ** 2)
        eds['var_of_sample_mean'][i][j] = eds['var'][i][j] / (num_journeys)

CPU times: user 6.03 ms, sys: 3.89 ms, total: 9.92 ms
Wall time: 9.44 ms


In [317]:
def random_walks(shop_num, num_steps):
    shop_list = list(range(shop_num))
    transition_matrix = grouped_rand_square_array(shop_num, groups=5, group_bonus=10)
    walk = [0]
    current_shop = 0
    last_visited = np.zeros(shop_num).astype(int)

    completed_journeys = {}
    for i in range(shop_num):
        completed_journeys[i] = {}
        for j in range(shop_num):
            completed_journeys[i][j] = {'num': 0, 'total_steps': 0, 'sum_of_sqares': 0}
            
    for steps in range(1, num_steps):
        current_shop = np.random.choice(shop_list, p=transition_matrix[current_shop])
        walk.append(current_shop)
        last_occurence = last_visited[current_shop]
        last_visited[current_shop] = steps
        for i in range(last_occurence, steps):
            completed_journeys[walk[i]][current_shop]['num'] += 1
            completed_journeys[walk[i]][current_shop]['total_steps'] += (steps - i)
            completed_journeys[walk[i]][current_shop]['sum_of_sqares'] += (steps - i) ** 2
        
    eds = {'n': {}, 'eds': {}, 'var': {}, 'var_of_sample_mean': {}}
    for i in range(shop_num):
        eds['n'][i] = {}
        eds['eds'][i] = {}
        eds['var'][i] = {}
        eds['var_of_sample_mean'][i] = {}
        for j in range(shop_num):
            num_journeys = completed_journeys[i][j]['num']
            total_steps = completed_journeys[i][j]['total_steps']
            sum_of_sqares = completed_journeys[i][j]['sum_of_sqares']
            eds['n'][i][j] = num_journeys
            eds['eds'][i][j] = total_steps / num_journeys
            eds['var'][i][j] = (sum_of_sqares / num_journeys) - (eds['eds'][i][j] ** 2)
            eds['var_of_sample_mean'][i][j] = eds['var'][i][j] / (num_journeys)
            
    return eds

In [319]:
shop_nums = [400, 550, 700]
time_elapsed = {}

for shop_num in shop_nums:
    start_time = time()
    eds = random_walks(shop_num, shop_num * 300)
    time_elapsed[shop_num] = time() - start_time
    print(f'{shop_num} shops: {time_elapsed[shop_num]} seconds')

400 shops: 127.20721197128296 seconds
550 shops: 223.08397388458252 seconds
700 shops: 362.36247205734253 seconds


In [327]:
pd.DataFrame(eds['n']).T.iloc[600:610, 600:610]

,600,601,602,603,604,605,606,607,608,609
600,256,255,255,255,257,256,257,257,255,257
601,248,247,248,248,248,248,248,248,248,248
602,258,255,257,258,258,258,258,258,258,258
603,241,239,239,240,241,241,241,241,241,241
604,263,261,262,262,264,262,265,265,262,264
605,258,255,255,257,258,257,258,258,257,258
606,242,242,242,242,243,242,243,243,242,242
607,266,266,266,266,266,266,267,266,266,266
608,254,252,253,253,254,254,254,254,253,254
609,292,292,292,292,293,292,293,293,292,292


In [285]:
pd.DataFrame(eds).T.iloc[:10, :10]

KeyError: 'eds'

In [291]:
eds_matrix = get_eds_for_all_pairs(transition_matrix, max_steps=100)

/Users/arthurjackrussell/venv/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars


In [292]:
pd.DataFrame(eds_matrix).T.iloc[:10, :10]

,0,1,2,3,4,5,6,7,8,9
0,145.852320,70.396255,96.372607,105.981291,130.588395,72.223049,105.908651,39.175069,40.015308,46.711125
1,142.007588,65.893622,88.700702,97.330898,119.264261,47.778855,93.952114,39.783145,40.183161,47.554969
2,144.645579,40.808560,93.093598,101.047677,125.322765,53.141605,95.623667,39.660389,39.847979,48.155844
3,144.032873,51.109525,79.729911,102.074772,122.537420,55.072282,96.167332,39.532842,40.245652,45.811210
4,144.724955,44.701709,86.528034,87.964417,120.669799,54.835567,74.658057,40.261595,40.821420,48.344455
5,145.579786,64.522252,83.062526,94.990209,125.635192,67.928613,92.321014,39.674614,38.931074,47.685252
6,144.549074,57.755184,85.459669,86.061627,104.469975,57.761842,97.000414,39.377775,40.480196,47.835637
7,145.941463,73.102392,99.665635,108.231589,131.972239,74.419855,107.330488,36.735700,37.942883,45.416328
8,146.615909,73.584417,101.057374,108.441961,133.448125,74.404008,107.645409,35.526954,38.495354,46.155643
9,145.260546,73.746201,100.848258,108.429645,133.639625,74.442560,107.967219,36.487134,37.814614,46.188437


# Experiments

In [320]:
shop_nums = [400, 550, 700]
time_elapsed = {}

for shop_num in shop_nums:
    transition_matrix = rand_square_array(shop_num)
    start_time = time()
    eds_pairs = get_eds_for_all_pairs(transition_matrix)
    time_elapsed[shop_num] = time() - start_time
    print(f'{shop_num} shops: {time_elapsed[shop_num]} seconds')

/Users/arthurjackrussell/venv/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars


400 shops: 100.84437227249146 seconds
550 shops: 431.84464406967163 seconds
700 shops: 1410.7342820167542 seconds


In [154]:
time_elapsed

{40: 0.18504095077514648,
 80: 0.7998449802398682,
 120: 2.563930034637451,
 160: 5.7952048778533936,
 200: 10.865333080291748,
 240: 17.60831880569458,
 280: 27.063740968704224,
 320: 48.40631675720215,
 360: 96.50931882858276,
 400: 128.72253012657166}

In [163]:
pd.DataFrame(eds_pairs).isna().any().any()

False

# Questions

Questions we might ask:

- What assumptions are made in the calculation of EDS?
- How would you calculate EDS if the number of shops was small (say <1,000)?
- What challenges would be faced as the number of shops grows? How would you deal with these?
- What are the properties of EDS as a metric?
- Given a matrix of EDS values, how might you estimate the latent shopping preferences of the customers?

#### Assumptions
 - p(also shops at) doesn't tell us p(next shops at).
   - e.g. in a world with three supermarkets and three florists, it may be that each person has their preference of each. However they will visit the supermarket more often than the florist.
   - this simplification may give us desired properties however - frequency doesn't necessarily correlate with loyalty/affinity
 - I'm assuming you can go back and visit any shop multiple times before you are "absorbed" by your first visit to shop B

#### How to calculate
 - Matrix multiplication as demonstrated here - achieve steady state then calculate infinite sums - seems very accurate given empirical testing

#### What challenges would be faced as the number of shops grows? How would you deal with these?
 - The compute time scales as O(n^?)